In [ ]:
from astropy.io import fits
import glob
from desitarget.sv1.sv1_targetmask import bgs_mask as bgs_mask_sv1
from desitarget.sv2.sv2_targetmask import bgs_mask as bgs_mask_sv2
from desitarget.sv3.sv3_targetmask import bgs_mask as bgs_mask_sv3

from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
from desispec.coaddition import coadd_cameras
from astropy.table import Table, vstack, hstack, join


In [ ]:
sv1_bgs_bits = '|'.join([_ for _ in bgs_mask_sv1.names() if 'BGS' in _])
sv2_bgs_bits = '|'.join([_ for _ in bgs_mask_sv2.names() if 'BGS' in _])
sv3_bgs_bits = '|'.join([_ for _ in bgs_mask_sv3.names() if 'BGS' in _])

## Everest

There are three types of data, `perexp`, `pernight`, and `cumulative`.  Each has its own subdirectory structure.

### Spectra
While spectra are saved in different files, the `coadd*.fits` provide a common format for the spectra we want from all three subdirectories.

HDUs inside the fits file are 'FIBERMAP', 'EXP_FIBERMAP', 'X_WAVELENGTH', 'X_FLUX', 'X_IVAR', 'X_MASK', 'X_RESOLUTION', 'SCORES' where X \in B, R, Z

FIBERMAP contains metadata that we want to save.



### Redshifts
Redshift information is in redrock files.

HDUs inside the fits file are 'REDSHIFTS', 'FIBERMAP', 'EXP_FIBERMAP', 'TSNR2'

In [ ]:
rootdir="/global/project/projectdirs/desi/spectro/redux/everest/tiles/"
subdir="pernight/" #"perexp/"
allcoadd_files = glob.glob(rootdir+subdir+'/*/*/coadd-*-*-*.fits')

In [ ]:
for name in allcoadd_files:
    
    #scrub path for information
    dum = name.split('/')
    sd1 = dum[-3]
    sd2 = dum[-2]
    subname = name[name.find('coadd')+6:-5]
    dum = subname.split('-')
    panel=dum[0]
    
    zname=name.replace('coadd','redrock')
    
    #Read spectra and coadd cameras, read 
    
    pspectra = read_spectra(name)
    cspectra = coadd_cameras(pspectra)
    zbest = Table.read(zname, 'redshifts')

    #Make selection: only targets

    select = (cspectra.fibermap['OBJTYPE'] == 'TGT') & (cspectra.fibermap['COADD_FIBERSTATUS'] == 0)
    
    #Access the relevant keys
    
    tid = cspectra.fibermap[select]['TARGETID']
    tra = cspectra.fibermap[select]['TARGET_RA']
    tdec = cspectra.fibermap[select]['TARGET_DEC']
    otype = cspectra.fibermap[select]['OBJTYPE']
        
    desi_target = cspectra.fibermap[select]['DESI_TARGET']
    mws_target = cspectra.fibermap[select]['MWS_TARGET']
    bgs_target = cspectra.fibermap[select]['BGS_TARGET']
    
    
    try:
        target_prefix='SV3_'
        scnd_target = cspectra.fibermap[select][target_prefix+'SCND_TARGET']
    except:
        try:
            scnd_target = cspectra.fibermap[select]['SCND_TARGET']
        except:
            print('No secondary targets column')
            # If you're appending the spectra from all files into 1 giant array, you'll want to
            # Fill in the scnd_target column here with 'NULL' or something
            
    
    zs = zbest[select]['Z'],
    zerr = zbest[select]['ZERR']
    zwarn = zbest[select]['ZWARN']
    spectype = zbest[select]['SPECTYPE']
    subtype = zbest[select]['SUBTYPE']
    deltachi2 = zbest[select]['DELTACHI2']
        
    
    # Access spectra here
    
    allwave = cspectra.wave['brz']
    allflux = cspectra.flux['brz'][select]
    allivar = cspectra.ivar['brz'][select]
    allmask = cspectra.mask['brz'][select]

    

In [ ]:
    target_prefix=''

    try:
        sv_target = cspectra.fibermap[select]['SV3_DESI_TARGET']
        target_prefix='SV3_'
        print('There is SV3')
    except:
        try:
            sv_target = cspectra.fibermap[select]['SV2_DESI_TARGET']
            target_prefix='SV2_'
            print('There is SV2')
        except:
            try:
                sv_target = cspectra.fibermap[select]['SV1_DESI_TARGET']
                target_prefix='SV1_'
                print('There is SV1')
            except:
                sv_target = cspectra.fibermap[select]['SV1_DESI_TARGET']
                print("No SV target bits found")
                
    desi_target = cspectra.fibermap[select][target_prefix+'DESI_TARGET']
    mws_target = cspectra.fibermap[select][target_prefix+'MWS_TARGET']
    bgs_target = cspectra.fibermap[select][target_prefix+'BGS_TARGET']
    scnd_target = cspectra.fibermap[select][target_prefix+'SCND_TARGET']